In [102]:
import math
from web3 import Web3, AsyncWeb3
from web3 import eth
import abi as abi_import
import time
from datetime import datetime

In [103]:
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/HsRAh-rTKhWRIzJAVCfUU3VXFqilsfLf'))
w3.is_connected()

True

In [104]:
sy_addr = '0xED97f94dd94255637A054098604E0201C442a3FD'
sy_abi = abi_import.sy_token_abi

mkt_abi = abi_import.mkt
mkt_addr = '0xC9beCdbC62efb867cB52222b34c187fB170379C6'


In [105]:
contract_instance = w3.eth.contract(address=mkt_addr, abi=mkt_abi)

In [106]:
mkt_data = contract_instance.functions._storage().call()

In [107]:
total_pt = mkt_data[0]
total_sy = mkt_data[1]
impliedRate = mkt_data[2]

In [108]:
pt_eth =  float(Web3.from_wei(total_pt, 'ether'))
sy_eth =  float(Web3.from_wei(total_sy, 'ether'))

In [109]:
pt_ratio = pt_eth / (pt_eth + sy_eth)
px_ratio = pt_ratio / (1-pt_ratio)

In [110]:
ln_lastImpliedRate = float(Web3.from_wei(impliedRate, 'ether'))
lastImpliedRate = math.exp(ln_lastImpliedRate)

In [111]:
scalar_root =  contract_instance.functions.scalarRoot().call()

In [112]:
initAnchor = contract_instance.functions.initialAnchor().call()

In [113]:
initAnchor_eth = float(Web3.from_wei(initAnchor, 'ether'))
scalar_root_eth = float(Web3.from_wei(scalar_root, 'ether'))

In [114]:
startTime = datetime(2023, 10, 11, 3, 19, 11).timestamp()
startTime

1697008751.0

In [115]:
expiry = contract_instance.functions.expiry().call()
unixYear = 31556926

In [116]:
(expiry - startTime) / unixYear

0.4618716347720307

In [117]:
expiry

1711584000

In [118]:
def update_now():
    return time.time()

def rateScalar(nowTime):
    return scalar_root_eth / (nowTime / unixYear)

def yrsLeft(nowTime):
    return (expiry - nowTime) / unixYear

def rateAnchor(nowTime):
    return lastImpliedRate**yrsLeft(nowTime) - (math.log(pt_ratio/(1-pt_ratio))) / rateScalar(nowTime)

In [119]:
rtScalar = rateScalar(update_now())
rtAnchor = rateAnchor(update_now())

In [120]:
exchange_rt_px = ((math.log(pt_ratio/(1-pt_ratio))) / rtScalar) + rtAnchor

In [121]:
pt_price = 1 / exchange_rt_px
sy_price = pt_price * ((lastImpliedRate)**(yrsLeft(update_now())) - 1)

In [122]:



silo_addr = '0x32a4Bcd8DEa5E18a12a50584682f8E4B77fFF2DF'

siloYield = '0x96eFdF95Cc47fe90e8f63D2f5Ef9FB8B180dAeB9'
yieldToken = '0xD533a949740bb3306d119CC777fa900bA034cd52' # crv
silo_contract = w3.eth.contract(address=silo_addr, abi=silo)


In [123]:
borrowAPY = silo_contract.functions.borrowAPY(siloYield, yieldToken).call()

In [124]:
borrowAPY_eth = float(Web3.from_wei(borrowAPY, 'ether'))
borrowAPY_eth

0.07308264088224

In [125]:
depositAPY = silo_contract.functions.depositAPY(siloYield, yieldToken).call()
depositAPY_eth = float(Web3.from_wei(depositAPY, 'ether'))
depositAPY_eth

0.03808975206798028

In [126]:
util = silo_contract.functions.getUtilization(siloYield, yieldToken).call()
util_eth = float(Web3.from_wei(util,'ether'))
util_eth

0.5211865332808175

In [127]:
liq = silo_contract.functions.liquidity(siloYield, yieldToken).call()
liq_eth = float(Web3.from_wei(liq,'ether'))
liq_eth

3894703.7511774716

In [128]:
borrow = silo_contract.functions.totalBorrowAmount(siloYield, yieldToken).call()
borrow_eth = float(Web3.from_wei(borrow,'ether'))
borrow_eth

4232390.415161393

In [129]:
deposits = silo_contract.functions.totalDeposits(siloYield, yieldToken).call()
deposits_eth = float(Web3.from_wei(deposits,'ether'))
deposits_eth

8120682.605743696

In [130]:
crvUSD = '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'

borrowAPY_crvUSD = float(Web3.from_wei(silo_contract.functions.borrowAPY(siloYield, crvUSD).call(), 'ether'))
depositAPY_crvUSD = float(Web3.from_wei(silo_contract.functions.depositAPY(siloYield, crvUSD).call(), 'ether'))
util_crvUSD = float(Web3.from_wei(silo_contract.functions.getUtilization(siloYield, crvUSD).call(), 'ether'))

liq_crvUSD= float(Web3.from_wei(silo_contract.functions.liquidity(siloYield, crvUSD).call(), 'ether'))

borrow_crvUSD = float(Web3.from_wei(silo_contract.functions.totalBorrowAmount(siloYield, crvUSD).call(), 'ether'))

deposits_crvUSD = float(Web3.from_wei(silo_contract.functions.totalDeposits(siloYield, crvUSD).call(), 'ether'))

In [131]:
print("borrow apy: ", borrowAPY_crvUSD)
print("deposit apy: ", depositAPY_crvUSD)
print("util: ", util_crvUSD)
print("liq: ", liq_crvUSD)
print("borrows: ", borrow_crvUSD)
print("deposits: ", deposits_crvUSD)

borrow apy:  0.070295828781888
deposit apy:  0.04941529230812298
util:  0.7029582881666394
liq:  11842434.001976823
borrows:  27922394.624218266
deposits:  39721268.09549636


In [132]:
interestRateModel = silo_contract.functions.getModel(siloYield, crvUSD).call()

In [133]:

model_contract = w3.eth.contract(address=interestRateModel, abi=model_abi)

In [134]:
dp = model_contract.functions.DP().call()
rcomp_max = model_contract.functions.RCOMP_MAX().call()
x_max = model_contract.functions.X_MAX().call()


In [135]:
config = model_contract.functions.getConfig(siloYield, crvUSD).call()

In [136]:
config # upot, ucrit, ulow, ki, kcrit, klow, klin, beta, ri, Tcrit

(800000000000000000,
 900000000000000000,
 600000000000000000,
 183506,
 237823439878,
 26424826653,
 3170979198,
 27777777777778,
 2230175119,
 0)

In [137]:
uopt = config[0]
ucrit = config[1]
ulow = config[2]
ki = config[3]
kcrit = config[4]
klow = config[5]
klin = config[6]
beta = config[7]
ri = config[8]
Tcrit = config[9]

config_tuple = (uopt, ucrit, ulow, ki, kcrit, klow, klin, beta, ri, Tcrit)

In [138]:
cb = beta * 3600
clin = klin * 365 * 24 * 3600 * uopt
clow = klow * 365 * 24 * 3600 * ulow
ccrit = 365 * 24 * 3600 * (1.0 - ulow)
ci = ki * 365 * (24**2) * (3600**2) * (1.0 - uopt)

In [157]:
print(currInterestRate(latestBlockTS(), update_now(), ri, util_crvUSD, Tcrit))

0.07029582880831842


In [ ]:
interestRateModel

In [ ]:
latestBlockTS()

In [158]:
currIntRate = model_contract.functions.getCurrentInterestRate(siloYield, crvUSD, expiry).call()

In [159]:
Web3.from_wei(currIntRate, 'ether')

Decimal('0.070295828781888')

In [162]:
Web3.from_wei(model_contract.functions.getCurrentInterestRate(siloYield, yieldToken, expiry).call(), 'ether')

Decimal('1.849035021233184')

In [163]:
Web3.from_wei(model_contract.functions.getCompoundInterestRate(siloYield, crvUSD, latestBlockTS()).call(), 'ether')


Decimal('0.000021559794339924')

In [164]:
Web3.from_wei(model_contract.functions.getCompoundInterestRate(siloYield, crvUSD, expiry).call(), 'ether')

Decimal('0.031762673327458142')

In [ ]:
0.031798680551912693 + 0.07033427280576

In [ ]:
sy_abi = '[{"inputs":[{"internalType":"string","name":"_name","type":"string"},{"internalType":"string","name":"_symbol","type":"string"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_collateralToken","type":"address"},{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_incentiveController","type":"address"},{"internalType":"address","name":"_siloLens","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"actualSharesOut","type":"uint256"},{"internalType":"uint256","name":"requiredSharesOut","type":"uint256"}],"name":"SYInsufficientSharesOut","type":"error"},{"inputs":[{"internalType":"uint256","name":"actualTokenOut","type":"uint256"},{"internalType":"uint256","name":"requiredTokenOut","type":"uint256"}],"name":"SYInsufficientTokenOut","type":"error"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"SYInvalidTokenIn","type":"error"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"SYInvalidTokenOut","type":"error"},{"inputs":[],"name":"SYZeroDeposit","type":"error"},{"inputs":[],"name":"SYZeroRedeem","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"user","type":"address"},{"indexed":false,"internalType":"address[]","name":"rewardTokens","type":"address[]"},{"indexed":false,"internalType":"uint256[]","name":"rewardAmounts","type":"uint256[]"}],"name":"ClaimRewards","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"caller","type":"address"},{"indexed":true,"internalType":"address","name":"receiver","type":"address"},{"indexed":true,"internalType":"address","name":"tokenIn","type":"address"},{"indexed":false,"internalType":"uint256","name":"amountDeposited","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amountSyOut","type":"uint256"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint8","name":"version","type":"uint8"}],"name":"Initialized","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"caller","type":"address"},{"indexed":true,"internalType":"address","name":"receiver","type":"address"},{"indexed":true,"internalType":"address","name":"tokenOut","type":"address"},{"indexed":false,"internalType":"uint256","name":"amountSyToRedeem","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amountTokenOut","type":"uint256"}],"name":"Redeem","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"user","type":"address"}],"name":"accruedRewards","outputs":[{"internalType":"uint256[]","name":"rewardAmounts","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"asset","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"assetInfo","outputs":[{"internalType":"enum IStandardizedYield.AssetType","name":"assetType","type":"uint8"},{"internalType":"address","name":"assetAddress","type":"address"},{"internalType":"uint8","name":"assetDecimals","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"claimOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"user","type":"address"}],"name":"claimRewards","outputs":[{"internalType":"uint256[]","name":"rewardAmounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"collateralToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"receiver","type":"address"},{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"uint256","name":"amountTokenToDeposit","type":"uint256"},{"internalType":"uint256","name":"minSharesOut","type":"uint256"}],"name":"deposit","outputs":[{"internalType":"uint256","name":"amountSharesOut","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"exchangeRate","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getRewardTokens","outputs":[{"internalType":"address[]","name":"rewardTokens","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getTokensIn","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getTokensOut","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"incentiveController","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"isValidTokenIn","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"isValidTokenOut","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lastRewardBlock","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pendingOwner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"uint256","name":"amountTokenToDeposit","type":"uint256"}],"name":"previewDeposit","outputs":[{"internalType":"uint256","name":"amountSharesOut","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint256","name":"amountSharesToRedeem","type":"uint256"}],"name":"previewRedeem","outputs":[{"internalType":"uint256","name":"amountTokenOut","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"receiver","type":"address"},{"internalType":"uint256","name":"amountSharesToRedeem","type":"uint256"},{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint256","name":"minTokenOut","type":"uint256"},{"internalType":"bool","name":"burnFromInternalBalance","type":"bool"}],"name":"redeem","outputs":[{"internalType":"uint256","name":"amountTokenOut","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"rewardIndexesCurrent","outputs":[{"internalType":"uint256[]","name":"indexes","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"rewardIndexesStored","outputs":[{"internalType":"uint256[]","name":"indexes","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"rewardState","outputs":[{"internalType":"uint128","name":"index","type":"uint128"},{"internalType":"uint128","name":"lastBalance","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"rewardToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"silo","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"siloLens","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"},{"internalType":"bool","name":"direct","type":"bool"},{"internalType":"bool","name":"renounce","type":"bool"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"unpause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"userReward","outputs":[{"internalType":"uint128","name":"index","type":"uint128"},{"internalType":"uint128","name":"accrued","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"yieldToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"stateMutability":"payable","type":"receive"}]'

sy_addr = '0xE05082B184a34668CD8A904D85FA815802BBb04c'


sy_contract = w3.eth.contract(address=sy_addr, abi=sy_abi)

In [ ]:
lastRewardBlock = sy_contract.functions.lastRewardBlock().call()

In [ ]:
w3.eth.get_block(lastRewardBlock)['timestamp']

In [ ]:
datetime.fromtimestamp(1697550419)

In [ ]:
crvUSD

In [152]:
sy_contract.functions.accruedRewards('0xC9beCdbC62efb867cB52222b34c187fB170379C6').call()

[0]

In [153]:
rewardsStored = float(Web3.from_wei(sy_contract.functions.rewardIndexesStored().call()[0], 'ether'))

In [154]:
rewardsStored * 10

0.023812711056301377

In [155]:
lastImpliedRate -1

0.07176347160928653

In [156]:
(rewardsStored * 10) + (lastImpliedRate - 1)


0.09557618266558791

In [167]:
1.0013136507479747 - 1

0.0013136507479747372

In [168]:
1 - 0.0013136507479747372

0.9986863492520253

In [169]:
1 / 1.0013136507479747

0.9986880726663484

In [170]:
2381271105630138 / 2370923560724211950840786

1.0043643519670307e-09

In [171]:
Web3.from_wei(2370923560724211950840786, 'ether')

Decimal('2370923.560724211950840786')

In [172]:
# 0.002381271105630138

delta = time.time() - startTime
delta / unixYear

0.018005433529842085

In [178]:
x = 0.002381271105630138 ** ((expiry - time.time()) / unixYear)

In [179]:
x

0.06849802771039565

In [180]:
2.322**(0.46)

1.4733184216331736

In [181]:
1/0.0307839

32.484513008423235

In [183]:
(1/0.0307839)*((expiry-time.time())/unixYear)

14.417461772658928

In [184]:
0.8490350212331841 / 14.417461772658928

0.05888935477139826

In [185]:
0.058889354**(1 / ((expiry-time.time())/unixYear))

0.001693178410266944

In [186]:
(expiry - time.time()) / unixYear

0.44382113940083906

In [187]:
1 / 0.4438211

2.2531601133880295

In [188]:
0.05888935477139826**(2.2531)

0.001693440355246964

In [189]:
0.01693440355246964 + 0.07177

0.08870440355246964

In [191]:
1.0717634716092865**(1/0.44)

1.170595114040302

In [192]:
1.0717634716092865**0.44

1.0309640901430779

In [194]:
1.07176 ** 0.44 - 1

0.03096262078210965

In [195]:
0.030964/0.03096262078210965

1.0000445446107438